In [ ]:
import os
from pathlib import Path
import zipfile
import shutil

# ===== 설정 =====
ROOT_DIR = Path("for_clean").resolve()

allowed_exts = {
    ".pdf",
    ".docx",
    ".xlsx",
    ".pptx",
    ".md", ".markdown",
    ".adoc", ".asciidoc",
    ".html", ".htm", ".xhtml",
    ".csv",
    ".png",
    ".jpg", ".jpeg",
    ".tif", ".tiff",
    ".bmp",
    ".webp",
    ".vtt",   # WebVTT
    ".hwp",
    ".hwpx",
}

print(f"[INFO] 정리 대상 루트 디렉터리: {ROOT_DIR}")

if not ROOT_DIR.is_dir():
    raise RuntimeError(f"{ROOT_DIR} 디렉터리가 존재하지 않습니다. 경로를 확인하세요.")


# ===== 1. ZIP 파일 안전하게 풀기 (중첩 ZIP 포함) =====
def safe_extract_zip(zip_path: Path, root: Path):
    """
    zip_path: 실제 .zip 파일 경로
    root: for_clean 절대 경로 (탈출 방지용)
    """
    print(f"[UNZIP] {zip_path}")
    with zipfile.ZipFile(zip_path, "r") as zf:
        for member in zf.infolist():
            # 일부 파이썬 버전에서는 ZipInfo에 is_dir() 없음 → 이름 기준
            is_dir = getattr(member, "is_dir", None)
            if callable(is_dir):
                is_dir = member.is_dir()
            else:
                is_dir = member.filename.endswith("/")

            # ★ 변경 1: 암호화된 파일은 아예 건너뛰기
            # flag_bits의 bit 0이 1이면 encrypted
            if member.flag_bits & 0x1:
                print(f"[SKIP ENCRYPTED] {member.filename} in {zip_path}")
                # zip 자체는 나중에 삭제되므로, 결과적으로 이 파일도 사라짐
                continue

            member_rel = Path(member.filename)

            # 절대 경로 및 부모로 빠져나가는 경로 방지 (zip slip 방어)
            if member_rel.is_absolute():
                member_rel = member_rel.relative_to(member_rel.anchor)

            dest = (zip_path.parent / member_rel).resolve()

            # for_clean 밖으로 나가면 무시
            if root not in dest.parents and dest != root:
                print(f"  [SKIP: 경로 탈출 감지] {member.filename} -> {dest}")
                continue

            if is_dir:
                dest.mkdir(parents=True, exist_ok=True)
            else:
                dest.parent.mkdir(parents=True, exist_ok=True)
                # ★ 변경 2: 개별 파일 추출 시 에러가 나면, dest 파일 삭제
                try:
                    with zf.open(member, "r") as src, open(dest, "wb") as dst:
                        shutil.copyfileobj(src, dst)
                except Exception as e:
                    # 압축 해제 중 에러 → 해당 파일 삭제
                    print(f"[DEL ERROR FILE] {dest} (압축 해제 중 에러: {e})")
                    try:
                        if dest.exists():
                            dest.unlink()
                    except Exception as e2:
                        print(f"[ERROR] 에러 파일 삭제 실패: {dest} ({e2})")
                    # 이 member는 건너뛰고 다음 파일 계속 진행
                    continue


def extract_all_zip_files(root: Path):
    """
    root 아래 모든 zip을 풀고, zip 파일은 삭제.
    중첩 zip이 있을 수 있으므로 zip이 더 이상 없을 때까지 반복.
    """
    while True:
        zip_paths = list(root.rglob("*.zip"))
        if not zip_paths:
            break

        for zpath in zip_paths:
            try:
                safe_extract_zip(zpath, root)
            except zipfile.BadZipFile:
                print(f"[WARN] 손상된 ZIP이거나 ZIP이 아님: {zpath} → 삭제 예정")
            except Exception as e:  # ★ 선택: 기타 예외도 잡아서 진행 계속
                print(f"[WARN] ZIP 처리 중 예외 발생: {zpath} ({e}) → ZIP 삭제")
            finally:
                # 어쨌든 zip 파일 자체는 지움 (allowed_exts에도 없으니)
                try:
                    zpath.unlink()
                    print(f"[DEL ZIP] {zpath}")
                except Exception as e:
                    print(f"[ERROR] ZIP 삭제 실패: {zpath} ({e})")


extract_all_zip_files(ROOT_DIR)


# ===== 2. 파일명 중복 제거 (처음 발견된 것만 유지) =====
def remove_duplicate_filenames(root: Path):
    """
    같은 파일명(확장자 포함)이 여러 번 나오면,
    os.walk 순서상 '처음' 발견된 것만 남기고 나머지는 삭제.
    """
    seen = set()  # 파일명(str)
    removed_count = 0

    for dirpath, dirnames, filenames in os.walk(root):
        # 결과를 어느 정도 예측 가능하게 하기 위해 정렬
        dirnames.sort()
        filenames.sort()

        for fname in filenames:
            if fname in seen:
                fpath = Path(dirpath) / fname
                try:
                    fpath.unlink()
                    removed_count += 1
                    print(f"[DEL DUP] {fpath}")
                except Exception as e:
                    print(f"[ERROR] 중복 파일 삭제 실패: {fpath} ({e})")
            else:
                seen.add(fname)

    print(f"[INFO] 중복 파일 삭제 개수: {removed_count}")


remove_duplicate_filenames(ROOT_DIR)


# ===== 3. 허용 확장자가 아닌 파일 삭제 =====
def remove_disallowed_exts(root: Path, allowed_exts: set[str]):
    removed_count = 0

    for path in root.rglob("*"):
        if not path.is_file():
            continue

        ext = path.suffix.lower()
        if ext not in allowed_exts:
            try:
                path.unlink()
                removed_count += 1
                print(f"[DEL EXT] {path} ({ext} 허용 안 됨)")
            except Exception as e:
                print(f"[ERROR] 확장자 필터 삭제 실패: {path} ({e})")

    print(f"[INFO] 허용되지 않은 확장자 파일 삭제 개수: {removed_count}")


remove_disallowed_exts(ROOT_DIR, allowed_exts)


# ===== 4. 비어 있는 디렉터리 삭제 =====
def remove_empty_dirs(root: Path):
    """
    하위 디렉터리부터 올라오면서 비어 있으면 삭제.
    root 디렉터리 자체는 남겨둔다.
    """
    removed_count = 0

    # bottom-up
    for dirpath, dirnames, filenames in os.walk(root, topdown=False):
        dpath = Path(dirpath)
        if dpath == root:
            continue

        # 안에 아무것도 없으면 삭제
        try:
            if not any(dpath.iterdir()):
                dpath.rmdir()
                removed_count += 1
                print(f"[DEL DIR] {dpath}")
        except Exception as e:
            print(f"[ERROR] 디렉터리 삭제 실패: {dpath} ({e})")

    print(f"[INFO] 삭제된 빈 디렉터리 개수: {removed_count}")


remove_empty_dirs(ROOT_DIR)

print("[DONE] 정리 완료")


In [1]:
import gdown

url = "https://drive.google.com/file/d/1woHO1uOyFNwfa032Tjr8ZJsUejKS7Jt4/view?usp=sharing"
output = "sample.zip"
gdown.download(url=url, output=output, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1woHO1uOyFNwfa032Tjr8ZJsUejKS7Jt4
From (redirected): https://drive.google.com/uc?id=1woHO1uOyFNwfa032Tjr8ZJsUejKS7Jt4&confirm=t&uuid=994128e4-e525-4de1-bad2-796a8b07676b
To: /Users/jeongho/git/python-playground/file_declutter/sample.zip
100%|██████████| 8.14G/8.14G [06:56<00:00, 19.5MB/s] 


'sample.zip'

In [20]:
import logging
import gdown
import zipfile
from pathlib import Path
from dataclasses import dataclass

logger = logging.getLogger(__name__)

URL = "https://drive.google.com/file/d/1blablablabla/view?usp=sharing"
OUTPUT = Path("sample.zip")


@dataclass
class GDriveZipDownloader:
    url: str = URL
    output: Path = OUTPUT

    def run(self) -> Path:
        """
        Google Drive에서 zip 파일을 다운로드하고 압축을 해제한다.

        :return: 압축이 풀린 디렉터리의 Path
        """
        logger.info("[1/2] ZIP 파일 다운로드 중...")
        # gdown.download(url=str(self.url), output=str(self.output), fuzzy=True)

        extract_dir = self.output.with_suffix("")  # sample.zip -> sample
        extract_dir.mkdir(exist_ok=True)

        logger.info("[2/2] 압축 해제 중... -> %s", extract_dir)
        with zipfile.ZipFile(self.output, 'r') as zip_ref:
            zip_ref.extractall("./")

        logger.info("완료!")
        logger.info("- 다운로드 파일 : %s", self.output)
        logger.info("- 압축 해제 경로 : %s", extract_dir)

        return extract_dir


# if __name__ == "__main__":
#     logging.basicConfig(
#         level=logging.INFO,  # INFO 로그 출력
#         format="%(asctime)s [%(levelname)s] %(message)s",
#     )

downloader = GDriveZipDownloader()
downloader.run()


OSError: [Errno 63] File name too long: 'sample/03. [ßäÆßàíßå½ßäéßàíßå½] ßäïßà»ßå½ßäçßà⌐ßå½_ßäëßà│ßäåßàíßäÉßà│ ßäÉßà⌐ßå╝ßäÆßàíßå╕ßäïßà«ßå½ßäïßàºßå╝ ßäëßà╡ßäëßà│ßäÉßàªßå╖ ßäîßàóßäÇßà«ßäÄßà«ßå¿ßäïßà│ßå» ßäïßà▒ßäÆßàíßå½ ßäëßàÑßå»ßäÇßà¿(ISMP) ßäåßà╡ßå╛ ßäëßà╡ßå»ßäîßà│ßå╝(POC) ßäïßà¡ßå╝ßäïßàºßå¿_Γàó.ßäÇßà╡ßäëßà«ßå» ßäåßà╡ßå╛ ßäÇßà╡ßäéßà│ßå╝ ßäëßà«ßäÆßàóßå╝ßäçßàíßå╝ßäïßàíßå½_AIßäçßà«ßäçßà«ßå½.pptx'

In [21]:
import zipfile

with zipfile.ZipFile("sample.zip") as zf:
    for info in zf.infolist():
        print("NAME:", info.filename)
        print("FLAG_BITS:", hex(info.flag_bits))
        print("UTF8 FLAG:", bool(info.flag_bits & 0x800))
        print()

NAME: sample/
FLAG_BITS: 0x0
UTF8 FLAG: False

NAME: sample/03. [ßäÆßàíßå½ßäéßàíßå½] ßäïßà»ßå½ßäçßà⌐ßå½_ßäëßà│ßäåßàíßäÉßà│ ßäÉßà⌐ßå╝ßäÆßàíßå╕ßäïßà«ßå½ßäïßàºßå╝ ßäëßà╡ßäëßà│ßäÉßàªßå╖ ßäîßàóßäÇßà«ßäÄßà«ßå¿ßäïßà│ßå» ßäïßà▒ßäÆßàíßå½ ßäëßàÑßå»ßäÇßà¿(ISMP) ßäåßà╡ßå╛ ßäëßà╡ßå»ßäîßà│ßå╝(POC) ßäïßà¡ßå╝ßäïßàºßå¿_Γàó.ßäÇßà╡ßäëßà«ßå» ßäåßà╡ßå╛ ßäÇßà╡ßäéßà│ßå╝ ßäëßà«ßäÆßàóßå╝ßäçßàíßå╝ßäïßàíßå½_AIßäçßà«ßäçßà«ßå½.pptx
FLAG_BITS: 0x0
UTF8 FLAG: False

NAME: sample/AI ßäÇßà╡ßäçßàíßå½ ßäåßà«ßå½ßäÆßà¬ßäëßàóßå╝ßäÆßà¬ßå» ßäîßà╡ßäïßà»ßå½ ßäÉßà⌐ßå╝ßäÆßàíßå╕ßäëßàÑßäçßà╡ßäëßà│ ßäæßà│ßå»ßäàßàóßå║ßäæßà⌐ßå╖ ßäÇßà«ßäÄßà«ßå¿ßäçßàíßå╝ßäïßàíßå½ ßäïßàºßå½ßäÇßà«ßäëßàíßäïßàÑßå╕.pdf
FLAG_BITS: 0x0
UTF8 FLAG: False

NAME: sample/2025ßäéßàºßå½ ßäÆßàíßå½-ßäàßàíßäïßà⌐ßäëßà│ ßäâßà╡ßäîßà╡ßäÉßàÑßå»ßäîßàÑßå╝ßäçßà« ßäÆßàºßå╕ßäàßàºßå¿ßäëßàªßå½ßäÉßàÑ ßäÇßà⌐ßå╝ßäâßà⌐ßå╝ßäÆßàºßå╕ßäàßàºßå¿ßäÇßà¬ßäîßàª/
FLAG_BITS: 0x0
UTF8 FLAG: False

NAME: sample/2025ßäéßàºßå½ ßäÆßàíßå½-ßäàßàíßäïßà⌐ßäëßà│ ßäâßà╡ßäîßà╡ßäÉßàÑßå»ßäîßàÑßå╝ßäçßà« ßäÆßàºßå

In [2]:
import os

os.pathconf(str("./"), 'PC_NAME_MAX')

255